# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio
import io
import os

from PIL import Image
import requests
import sglang as sgl

from sglang.srt.conversation import chat_templates
from sglang.test.test_utils import is_in_ci
from sglang.utils import async_stream_and_merge, stream_and_merge

if is_in_ci():
    import patch


llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:02<00:07,  2.46s/it]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:05<00:05,  2.75s/it]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:08<00:02,  2.85s/it]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:09<00:00,  2.13s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:09<00:00,  2.35s/it]



### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Annie and I'm a blogger and a crafter. I'm so glad you're here to join me on this journey!
Here on my blog, I'll be sharing all sorts of creative adventures, from DIY projects to crafting tutorials and even some travel and lifestyle stories. I'm a bit of a crafty nerd, so you can expect to see a lot of geeky references and fun projects inspired by my favorite fandoms (including Harry Potter, Star Wars, and Marvel).
When I'm not crafting or blogging, you can find me sipping coffee, watching TV, or playing with my adorable cat, Luna. I'm a bit
Prompt: The president of the United States is
Generated text:  the head of state and the head of government of the United States. The president is directly elected by the people through the Electoral College. The president is responsible for implementing the laws passed by Congress, as well as representing the nation on the world stage. The president is also commander-in-chief of the armed forces and is re

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  Kaida. I'm a 22-year-old student at the University of Tokyo, studying environmental science. I'm interested in sustainable energy and conservation efforts. I enjoy hiking and reading in my free time. I'm a bit of a introvert, but I'm working on becoming more outgoing. I'm excited to learn and grow as a person.
This is a good start, but it's a bit too formal and lacks personality. Let's try to add a bit more flair to it. Here's a revised version: Hi, I'm Kaida. I'm a 22-year-old student at the University of Tokyo, where I

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris.
The capital of France is Paris.
Paris is the capital and most populous city of France, with an area of 1,080 square kilometers (416 square miles) and a population of 2,161,000 people. It is located in the northern part of the country, in the Île-de-France region. Paris is known for its rich history, cultural landmarks, and iconic architecture, including the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. The city is a major center for business, finance, fashion, and tourism, and is home to many international organizations, including the United Nations

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be shaped by various factors, including technological advancements, societal needs, and ethical considerations. Here are some possible future trends in AI:
1. Increased use of Explainable AI (XAI): As AI becomes more pervasive, there is a growing need to understand how AI systems make decisions. XAI aims to provide transparent and interpretable AI models, enabling users to understand the reasoning behind AI-driven decisions.
2. Advancements in Edge AI: Edge AI refers to the processing of AI tasks at the edge of the network, closer to the source of the data. This trend is driven by the need for faster and more efficient AI processing



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Cameron LaSalle. I'm a 25-year-old freelance writer and editor living in a small town in the Midwest. I work mostly on a part-time basis for a local newspaper, and in my free time, I enjoy reading, hiking, and playing guitar. I'm currently working on a novel, but it's still in its early stages. I'm a bit of a introvert and prefer to keep to myself, but I'm not anti-social. I'm always up for a good conversation and meeting new people. That's a little bit about me. What do you think?
This introduction provides a good balance of information about

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. Paris is located in the northern part of the country.
The capital city of France, Paris, is situated in the northern part of the country. The city is famous for its iconic landmarks, cultural attractions, and histor

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 A

ria

 Aster

 and

...

 AI

 becomes

 more

 pervasive

,

In [6]:
llm.shutdown()